# Homework 5

## Consider the bivariate normal distribution we introduced in the class. Let's make the assumptions that  $\mu_x = 0$, $\mu_y = \mu_y$, $\sigma_x = \sigma_x$, $\sigma_y = 1$, and $\rho=\rho$.

### Write down the conditional distributions of X and Y. Note that both of them would be normal distributions. (See Section 1.4.3 and 1.4.3.1 of the lecture note.)


\begin{align}
(X|Y=y) & \sim N\left( \rho \sigma_x (y-\mu_y),\quad \sigma_x^2(1-\rho^2) \right),\\
(Y|X=x) & \sim N\left( \mu_y + \rho \frac{x}{\sigma_x},\quad (1-\rho^2) \right).
\end{align}

### Write a Julia function of Gibbs sampler that allows users to choose: the number of sampling points $N$, the values of $\mu_y$, $\sigma_x$, $\rho$, and initial values of $(x_0, y_0)$, and the number of burn-in points. You should follow the steps in Section 1.3 of the lecture note to draw $(x_n, y_n), n=1,\ldots,N$.


In [7]:
using Random, Statistics, Distributions

# Define Gibbs sampling function
function Gibbs_sampling(; μy = 1.0, σx = 1.0, N = 100, rho = 0.1, init = [2, 2], skip = 0) # total length: 1+skip+N; "1" for initial value

    x = ones(1+skip+N) * init[1]
    y = ones(1+skip+N) * init[2]

    for n = 2:skip+N+1
        x[n] = rand(Normal(rho * σx * (y[n-1]-μy), σx * sqrt(1-rho^2)))  # mean=rho*y_{n-1}, variance=1-rho^2
        y[n] = rand(Normal(μy + rho * x[n]/σx, sqrt(1-rho^2))) 
    end

    return x[1+skip+1:end], y[1+skip+1:end]

end

# Draw 100,000 samples
x, y = Gibbs_sampling(μy = 0.0, σx = 1.0, N = 100000, rho = 0.5, init = [2, 2], skip = 100)

# Check if the statistics are close
[mean(x) std(x)] |> display
[mean(y) std(y)] |> display
cor(x,y) 

1×2 Matrix{Float64}:
 -0.00341245  1.0016

1×2 Matrix{Float64}:
 -0.00450104  1.00332

0.4998478063897412

### Draw a graph  similar to the one we showed in the class. Remember to add sliders for both $N$ and $\rho$.

In [11]:
using Interact
using Distributions, Plots, LinearAlgebra, Statistics

@manipulate for num=50:10000, corr=-0.99:0.1:0.99
    
    # Draw a graph given the parameters
    x, y = Gibbs_sampling(μy = 0.0, σx = 1.0, N = num, rho = corr, init = [2,2], skip = 100)  
  
    # Set the boundary of the graph
    xmin = minimum(x) - abs(0.1*minimum(x));  xmax = maximum(x) + abs(0.1*maximum(x));
    ymin = minimum(y) - abs(0.1*minimum(y));  ymax = maximum(y) + abs(0.1*maximum(y));
    
    layout = @layout [a{0.4w,0.4h} _
                      b{0.4w,0.4h} c{0.6w, 0.4h}]

    # Set the scheme of the graph
    default(fillcolor=:lightgrey, markercolor=:white, grid=false, legend=false)
    plot(layout=layout, link=:both, size=(400, 400),  margin=-10Plots.pt)

    # Plot the scatter point and histogram
    scatter!(x,y, subplot=2, framestyle =:box, xlims=(xmin, xmax), ylims=(ymin, ymax))
    histogram!(x, subplot=1, orientation=:v, framestyle=:none, bins=min(num, 100), 
               normalize=:none, xlims=(xmin, xmax))
    histogram!(y, subplot=3, orientation=:h, framestyle=:none, bins=min(num, 100), 
               normalize=:none, ylims=(ymin, ymax))
               
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["num"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 10000, :min => 50, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable(5025), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/yijie/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/yijie/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/yijie/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/Users/yijie/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/Users/yijie/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000000014c4a0010, Task (runnable) @0x000000014c4a0010), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 5628319584, 4606312372349080345)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"50\",\"51\",\"52\",\"53\",\"54\",\"55\",\"56\",\"57\",\"58\",\"59\",\"60\",\"61\",\"62\",\"63\",\"64\",\"65\",\"66\",\"67\",\"68\",\"